In [ ]:
# house_type = ['modern',
#               'character',
#               'modern', 
#               'apartment',
#               'nan',
#               'character',
#               'special',
#               'special',
#               'duplex',
#               'apartment',
#               'special',
#               'modern',
#               'bungalow',
#               'bungalow',
#               'special',
#               'character']

# for i, item in enumerate(set(house_type)):
#     print(i, ' ', item)

In [ ]:
# facing = ['South',
#           'West',
#           'South',
#           'SW',
#           'nan',
#           'South',
#           'South',
#           'South',
#           'North',
#           'NW',
#           'North',
#           'South',
#           'ENE',
#           'East',
#           'West',
#           'North']

# for i, item in enumerate(set(facing)):
#     print(i, ' ', item)

In [ ]:
# region = ['YVR', 'nan', 'WYJ']
# for i, item in enumerate(set(region)):
#     print(i, ' ', item)

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.cluster import  DBSCAN

In [3]:
ROOT_DIR = '/content/gdrive/My Drive/Colab Notebooks'
data_dir = os.path.join(ROOT_DIR, 'preprocessed_data')

In [4]:
redident_data_file = os.path.join(data_dir, 'station_properties_v2.csv')
resident_df = pd.read_csv(redident_data_file, index_col='id', low_memory=False)

FileNotFoundError: ignored

In [ ]:
resident_df_tmp = resident_df.drop(['first_reading', 'last_reading'], axis=1)
resident_df_tmp.head()

,house_type,facing,region,RUs,EVs,FAGF,HP,FPG,FPE,IFRHG,NAC,FAC,PAC,BHE,IFRHE,WHIR
id,,,,,,,,,,,,,,,,
3,modern,South,YVR,2,0,0,0,0,0,1,1,0,0,1,0,0
4,character,West,YVR,1,0,1,0,0,0,1,1,0,0,0,0,0
5,modern,South,YVR,1,0,0,0,0,0,1,1,0,0,0,0,0
6,apartment,SW,YVR,0,0,0,0,0,0,0,1,0,0,1,0,0
7,special,South,YVR,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
resident_df_tmp.iloc[7]

house_type    special
facing          South
region            YVR
RUs                 0
EVs                 0
FAGF                1
HP                  0
FPG                 0
FPE                 0
IFRHG               0
NAC                 1
FAC                 0
PAC                 0
BHE                 0
IFRHE               0
WHIR                0
Name: 10, dtype: object

In [ ]:
house_type = pd.get_dummies(resident_df_tmp['house_type'], prefix='house_type')
house_type.head()

,house_type_apartment,house_type_bungalow,house_type_character,house_type_duplex,house_type_modern,house_type_special
id,,,,,,
3,0,0,0,0,1,0
4,0,0,1,0,0,0
5,0,0,0,0,1,0
6,1,0,0,0,0,0
7,0,0,0,0,0,1


In [ ]:
facing = pd.get_dummies(resident_df_tmp['facing'], prefix='facing')
facing.head()

,facing_ENE,facing_East,facing_NW,facing_North,facing_SW,facing_South,facing_West
id,,,,,,,
3,0,0,0,0,0,1,0
4,0,0,0,0,0,0,1
5,0,0,0,0,0,1,0
6,0,0,0,0,1,0,0
7,0,0,0,0,0,1,0


In [ ]:
region = pd.get_dummies(resident_df_tmp['region'], prefix='region')
region.head()

,region_WYJ,region_YVR
id,,
3,0,1
4,0,1
5,0,1
6,0,1
7,0,1


In [ ]:
resident_df_tmp = resident_df_tmp.drop(['house_type','facing','region'], axis=1)
resident_df_tmp.head()

,RUs,EVs,FAGF,HP,FPG,FPE,IFRHG,NAC,FAC,PAC,BHE,IFRHE,WHIR
id,,,,,,,,,,,,,
3,2,0,0,0,0,0,1,1,0,0,1,0,0
4,1,0,1,0,0,0,1,1,0,0,0,0,0
5,1,0,0,0,0,0,1,1,0,0,0,0,0
6,0,0,0,0,0,0,0,1,0,0,1,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
resident_df_tmp = resident_df_tmp.join([house_type, facing, region])
resident_df_tmp.head()

,RUs,EVs,FAGF,HP,FPG,FPE,IFRHG,NAC,FAC,PAC,BHE,IFRHE,WHIR,house_type_apartment,house_type_bungalow,house_type_character,house_type_duplex,house_type_modern,house_type_special,facing_ENE,facing_East,facing_NW,facing_North,facing_SW,facing_South,facing_West,region_WYJ,region_YVR
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,2,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
4,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1
5,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
6,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1


In [ ]:
clustering = DBSCAN(eps=2, min_samples=2).fit(resident_df_tmp.values)
clustering.labels_

array([ 0,  0,  0,  1,  0,  0,  0,  0, -1,  1,  0,  0, -1, -1,  0, -1])

In [ ]:
from sklearn.cluster import OPTICS
clustering = OPTICS(min_samples=2).fit(resident_df_tmp.values)
clustering.labels_

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/_optics.py:802: RuntimeWarning: divide by zero encountered in true_divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


array([ 0,  1,  0,  2,  1,  1,  1,  1, -1,  2,  1,  0, -1, -1,  1, -1])

In [ ]:
cluster_dict = {-1:[], 0:[], 1:[], 2:[]}

for house_id, group_id in zip(resident_df_tmp.index, clustering.labels_):
    print('id: ', house_id, '| group: ', group_id)
    cluster_dict[group_id].append(house_id)

id:  3 | group:  0
id:  4 | group:  1
id:  5 | group:  0
id:  6 | group:  2
id:  7 | group:  1
id:  8 | group:  1
id:  9 | group:  1
id:  10 | group:  1
id:  11 | group:  -1
id:  12 | group:  2
id:  13 | group:  1
id:  14 | group:  0
id:  15 | group:  -1
id:  18 | group:  -1
id:  19 | group:  1
id:  20 | group:  -1


In [ ]:
print('cluster 0 :', cluster_dict[0])
print('cluster 1 :', cluster_dict[1])
print('cluster 2 :', cluster_dict[2])
print('noise :', cluster_dict[-1])

cluster 0 : [3, 5, 14]
cluster 1 : [4, 7, 8, 9, 10, 13, 19]
cluster 2 : [6, 12]
noise : [11, 15, 18, 20]


ID 18 is not included.